In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from time import time
import pickle
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.metrics import r2_score
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.losses import MeanSquaredLogarithmicError
from tensorflow.keras.optimizers import Adam
%matplotlib inline

In [ ]:
my_ds=pd.read_csv('../input/web-scraped/used_cars.csv')

In [ ]:
ng_ds=pd.read_csv('../input/web-scraped/train-data.csv')

In [ ]:
ng_ds.rename(columns={'Price':'Price in Lakhs'},inplace=True)

In [ ]:
ng_c_ds=ng_ds[['Name','Location','Year','Kilometers_Driven','Fuel_Type','Owner_Type','Price in Lakhs']]

In [ ]:
my_ds

In [ ]:
list=[]
for i in my_ds['Price in Lakhs']:
    list.append(i.split(",")[0]+"."+i.split(",")[1])

In [ ]:
my_ds['Price in Lakhs']=list

In [ ]:
my_ds

In [ ]:
my_ds

In [ ]:
owner={'1st':'First','2nd':'Second','3rd':'Third','4th':'Fourth'}
my_ds['Owner_Type']=my_ds['Owner_Type'].map(owner)

In [ ]:
my_ds

In [ ]:
combined_ds=pd.concat([my_ds,ng_c_ds],ignore_index=True)

In [ ]:
combined_ds

In [ ]:
age=2021-combined_ds['Year']
combined_ds.insert(3,'Age',age)

In [ ]:
combined_ds['Fuel_Type'].value_counts()

In [ ]:
fuel={'Petrol':'Petrol','Diesel':'Diesel','Petrol + CNG':'CNG','CNG':'CNG','Petrol + LPG':'LPG','LPG':'LPG','Electric':'Electric'}
combined_ds['Fuel_Type']=combined_ds['Fuel_Type'].map(fuel)

In [ ]:
#dummy=pd.get_dummies(combined_ds['Fuel_Type'],drop_first=True)
#combined_ds=pd.concat([combined_ds,dummy],axis=1)

In [ ]:
combined_ds

In [ ]:
manufacturer= [i.split()[0] for i in combined_ds['Name']]

In [ ]:
combined_ds.insert(1,'Manufacturer',manufacturer)

In [ ]:
combined_ds

In [ ]:
combined_ds['Manufacturer'].value_counts()

In [ ]:
names=combined_ds['Manufacturer'].value_counts()
names_less_than_100=names[names<=100]
combined_ds['Manufacturer']=combined_ds['Manufacturer'].apply(lambda x: 'Other' if x in names_less_than_100 else x)

In [ ]:
combined_ds.info()

In [ ]:
combined_ds["Kilometers_Driven"] = [float(str(i).replace(",", "")) for i in combined_ds["Kilometers_Driven"]]

In [ ]:
combined_ds['Price in Lakhs']=combined_ds['Price in Lakhs'].astype(str).astype(float)

In [ ]:
km_driven_mean = combined_ds['Kilometers_Driven'].mean()
combined_ds['Kilometers_Driven']=[i if i<500000 else float(int(km_driven_mean)) for i in combined_ds['Kilometers_Driven']]

In [ ]:
combined_ds.corr()['Price in Lakhs'].sort_values(ascending=True)

In [ ]:
combined_ds

In [ ]:
#owner=pd.get_dummies(combined_ds['Owner_Type'],drop_first=True)
#combined_ds=pd.concat([combined_ds,owner],axis=1)

In [ ]:
#combined_ds

In [ ]:
#age_array=combined_ds['Age'].values
#kms_array=combined_ds['Kilometers_Driven'].values
#price_array=combined_ds['Price in Lakhs'].values

In [ ]:
#Age=std_scaler.fit_transform(age_array.reshape(-1,1))
#combined_ds['Age']=pd.DataFrame(Age)
#
#Kilometers_Driven=std_scaler.fit_transform(kms_array.reshape(-1,1))
#combined_ds['Kilometers_Driven']=pd.DataFrame(Kilometers_Driven)

#Price=std_scaler.fit_transform(price_array.reshape(-1,1))
#combined_ds['Price in Lakhs']=pd.DataFrame(Price)

In [ ]:
#manufacturer=pd.get_dummies(combined_ds['Manufacturer'],drop_first=True)
#combined_ds=pd.concat([combined_ds,manufacturer],axis=1)

In [ ]:
#location=pd.get_dummies(combined_ds['Location'],drop_first=True)
#combined_ds=pd.concat([combined_ds,location],axis=1)

In [ ]:
combined_ds['Manufacturer'].value_counts()

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder=LabelEncoder()

In [ ]:
combined_ds['Manufacturer'] = label_encoder.fit_transform(combined_ds['Manufacturer'])
combined_ds['Location'] = label_encoder.fit_transform(combined_ds['Location'])
combined_ds['Fuel_Type'] = label_encoder.fit_transform(combined_ds['Fuel_Type'])
combined_ds['Owner_Type'] = label_encoder.fit_transform(combined_ds['Owner_Type'])

In [ ]:
combined_ds

In [ ]:
#X=combined_ds.drop(['Name','Year','Price in Lakhs'],axis=1)
#y=combined_ds['Price in Lakhs']

In [ ]:
final_ds=pd.read_csv('../input/used-cars-cleaned1/Used_Cars_Cleaned1.csv')

In [ ]:
final_ds.head()

In [ ]:
std_scaler=StandardScaler()

In [ ]:
age_array=final_ds['Age'].values
kms_array=final_ds['Kilometers_Driven'].values
mileage_array=final_ds['Mileage'].values
engine_array=final_ds['Engine(CC)'].values
power_array=final_ds['Power(bhp)'].values
owner_array=final_ds['Owner_Type'].values

In [ ]:
Age=std_scaler.fit_transform(age_array.reshape(-1,1))
final_ds['Age']=pd.DataFrame(Age)
Kilometers_Driven=std_scaler.fit_transform(kms_array.reshape(-1,1))
final_ds['Kilometers_Driven']=pd.DataFrame(Kilometers_Driven)
Mileage=std_scaler.fit_transform(mileage_array.reshape(-1,1))
final_ds['Mileage']=pd.DataFrame(Mileage)
Engine=std_scaler.fit_transform(engine_array.reshape(-1,1))
final_ds['Engine(CC)']=pd.DataFrame(Engine)
Power=std_scaler.fit_transform(power_array.reshape(-1,1))
final_ds['Power(bhp)']=pd.DataFrame(Power)
Owner=std_scaler.fit_transform(owner_array.reshape(-1,1))
final_ds['Owner_Type']=pd.DataFrame(Owner)

In [ ]:
final_ds.head()

In [ ]:
dfnew = final_ds.select_dtypes(include=np.float)
dfnew

In [ ]:
X=final_ds.drop(['Year','Price in Lakhs'],axis=1)
y=final_ds['Price in Lakhs']

In [ ]:
X_train,X_valid,y_train,y_valid= train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
X_train.head()

In [ ]:
y_train

## Using Lasso

In [ ]:
start = time()

In [ ]:
from sklearn.linear_model import Lasso
lasso=Lasso()
lasso.fit(X_train,y_train)

In [ ]:
result_lasso=lasso.predict(X_valid)
mse_lasso=print("Mean Squared Error: %f" % mean_squared_error(y_valid,result_lasso, squared=False))
sq_lasso=print("R2 score : %f" % r2_score(y_valid,result_lasso))

In [ ]:
print(f'Time taken to run: {time() - start} seconds')

In [ ]:
plt.figure(figsize=(12,6))
plt.scatter(x=y_valid,y=result_lasso)
plt.xlabel('Actual Price')
plt.ylabel('Predicted Price')

## Using KNN

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
model = KNeighborsRegressor(n_neighbors=8)
model.fit(X_train,y_train)

In [ ]:
result_knn=model.predict(X_valid)
mse_knn=print("Mean Squared Error: %f" % mean_squared_error(y_valid,result_knn, squared=False))
sqknn=print("R2 score : %f" % r2_score(y_valid,result_knn))

In [ ]:
plt.figure(figsize=(12,6))
plt.scatter(x=y_valid,y=result_knn)
plt.xlabel('Actual Price')
plt.ylabel('Predicted Price')

## Using Randomized Seach CV

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
rfr=RandomForestRegressor()

In [ ]:
param_grid={'n_estimators':[int(x) for x in np.linspace(start=200,stop=2000,num=10)],
            'max_features':['auto','sqrt'],
            'max_depth':[int(x) for x in np.linspace(start=10,stop=100,num=11)],
            'min_samples_leaf':[1,2,3],
            'min_samples_split':[2,5,10]}

In [ ]:
rf_random=RandomizedSearchCV(rfr,param_grid,cv=3,verbose=2,random_state=42)

In [ ]:
start=time()

In [ ]:
rf_random.fit(X_train,y_train)

In [ ]:
bruh=time() - start

In [ ]:
print(f'Time taken to run: {bruh} seconds')

In [ ]:
print(rf_random.best_params_)
print(" ")
best_random=rf_random.best_estimator_
print(best_random)

In [ ]:
result_cv=best_random.predict(X_valid)
mse_rf=print("Mean Squared Error: %f" % mean_squared_error(y_valid,result_cv, squared=False))
sqrf=print("R2 score : %f" % r2_score(y_valid,result_cv))

In [ ]:
plt.figure(figsize=(12,6))
plt.scatter(x=y_valid,y=result_cv)
plt.xlabel('Actual Price')
plt.ylabel('Predicted Price')

## Using SVR

In [ ]:
from sklearn.svm import SVR
regressor = SVR(kernel = 'rbf')
regressor.fit(X_train, y_train)

In [ ]:
pred_svr=regressor.predict(X_valid)

In [ ]:
mse_svr=print("Mean Squared Error: %f" % mean_squared_error(y_valid,pred_svr, squared=False))
sqsvr=print("R2 score : %f" % r2_score(y_valid,pred_svr))

In [ ]:
plt.figure(figsize=(12,6))
plt.scatter(x=y_valid,y=pred_svr)
plt.xlabel('Actual Price')
plt.ylabel('Predicted Price')

## Using NN

In [ ]:
hidden_units1 = 160
hidden_units2 = 480
hidden_units3 = 256
learning_rate = 0.01
# Creating model using the Sequential in tensorflow
def build_model_using_sequential():
    model= Sequential([
    Dense(hidden_units1, kernel_initializer='normal', activation='relu'),
    Dropout(0.2),
    Dense(hidden_units2, kernel_initializer='normal', activation='relu'),
    Dropout(0.2),
    Dense(hidden_units3, kernel_initializer='normal', activation='relu'),
    Dense(1, kernel_initializer='normal', activation='linear')
    ])
    return model
# build the model
model = build_model_using_sequential()

In [ ]:
start=time()

In [ ]:
# loss function
msle = MeanSquaredLogarithmicError()
model.compile(
    loss=msle, 
    optimizer=Adam(learning_rate=learning_rate), 
    metrics=[msle]
)
# train the model
history = model.fit(
    X_train.values, 
    y_train.values, 
    epochs=50, 
    batch_size=64,
    validation_split=0.2
)

In [ ]:
print(f'Time taken to run: {time() - start} seconds')

In [ ]:
predictionNN=model.predict(X_valid)

In [ ]:
mse_nn=print("Mean Squared Error: %f" % mean_squared_error(y_valid,predictionNN, squared=False))
sqnn=print("R2 score : %f" % r2_score(y_valid,predictionNN))

In [ ]:
plt.figure(figsize=(12,6))
plt.scatter(x=y_valid,y=predictionNN)
plt.xlabel('Actual Price')
plt.ylabel('Predicted Price')

In [ ]:
def plot_history(history, key):
  plt.plot(history.history[key])
  plt.plot(history.history['val_'+key])
  plt.xlabel("Epochs")
  plt.ylabel(key)
  plt.legend([key, 'val_'+key])
  plt.show()
# Plot the history
plot_history(history, 'mean_squared_logarithmic_error')

In [ ]:
scores=[4.97, 4.212, 3.6065, 3.611]
xax=[0, 1, 2, 3]

fig, axes=plt.subplots(figsize=(20,10))

axes.bar(xax, scores, width=0.5, color=['k','r', 'g', 'b'], alpha=0.7)

axes.set_xticks(xax)
axes.set_xticklabels(['Support Vector Regression','K-Nearest Neigbours','Random Forest', 'Neural Network'])
axes.set_ylabel('MSE SCORE')

In [ ]:
scores=[0.7,0.825, 0.881,0.894]
xax=[0, 1, 2, 3]

fig, axes=plt.subplots(figsize=(20,10))

axes.bar(xax, scores, width=0.5, color=['k','r', 'b', 'g'], alpha=0.7)

axes.set_xticks(xax)
axes.set_xticklabels(['Support Vector Regression', 'K-Nearest Neigbours','Random Forest ', 'Neural Network'])
axes.set_ylabel('R-Squared Score')

In [ ]:
scores=[501.2, 8.7]
xax=[0, 1]

fig, axes=plt.subplots(figsize=(20,10))

axes.bar(xax, scores, width=0.5, color=['r', 'g'], alpha=0.7)

axes.set_xticks(xax)
axes.set_xticklabels(['Random Forest ', 'Neural Network'])
axes.set_ylabel('Exceution Time (s)')

In [ ]:
#filename = 'finalized_model.sav'
#pickle.dump(rf_random, open(filename, 'wb'))

In [ ]:
models_scores_table = pd.DataFrame({'MSE':[4.976465,4.212559,3.618779,3.627327],'R^2':[0.798655,,,,,0.893616]}, index=['SVR','KNN','Random Forest','Neural Network'])

In [ ]:
models_scores_table

In [ ]:
models_scores_table = pd.DataFrame({'SVR':[4.976465,0.798655],'KNN':[4.212559,0.855725],'RandomForest':[3.618779,0.892942],'Neural Network':[3.627327,0.893616]}, index=['MSE','R^2'])

In [ ]:
models_scores_table